In [1]:
import os
import sys


import matplotlib.pyplot as plt
%matplotlib inline
import numpy as np
import pickle

ModuleNotFoundError: No module named 'matplotlib'

In [ ]:
from sklearn.inspection import permutation_importance

In [ ]:
from sklearn.linear_model import LinearRegression

In [ ]:
from sklearn.metrics import mean_squared_error

In [ ]:
from sklearn.model_selection import train_test_split

In [ ]:
from sklearn.feature_selection import SelectFromModel

In [ ]:
from sklearn.ensemble import RandomForestRegressor

In [ ]:
from statsmodels.regression.linear_model import OLS

In [ ]:
module_path = os.path.abspath(os.path.join('..'))
if module_path not in sys.path:
    sys.path.insert(0, module_path)

In [ ]:
from spectroscopy.utils import load_training_data, get_wavelength_columns
from spectroscopy.model import score_model

# modeling

note: "I think the best is the ground samples from South"

# load in data

In [ ]:
df = load_training_data()

In [ ]:
df.head()

In [ ]:
df.columns

In [ ]:
feature_columns = get_wavelength_columns(df)

In [ ]:
X, y = df[feature_columns], df['Ammonia-N']

In [ ]:
y.value_counts(dropna=False)

In [ ]:
y.describe()

In [ ]:
y.hist()

In [ ]:
X.describe()

In [ ]:
len(X.columns)

this is too many columns, too many features for linear regression. lets try a tree based model

In [ ]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, random_state=10)

In [ ]:
len(X_test), len(y_test)

In [ ]:
len(X_train), len(y_train)

In [ ]:
y_test.hist()

In [ ]:
y_train.hist()

# fit baseline model to raw data

In [ ]:
model = RandomForestRegressor(random_state=10, max_depth=5, n_estimators=10)

In [ ]:
model.fit(X_train, y_train)

# score model

In [ ]:
num_features = 10

In [ ]:
# feature_names = X_train.columns
# result = permutation_importance(model, X_train, y_train, n_repeats=10,
#                                 random_state=42)
# perm_sorted_idx = result.importances_mean.argsort()

# tree_importance_sorted_idx = np.argsort(model.feature_importances_)
# tree_indices = np.arange(0, len(model.feature_importances_)) + 0.5

# fig, (ax1, ax2) = plt.subplots(1, 2, figsize=(12, 8))
# ax1.barh(tree_indices,
#          model.feature_importances_[tree_importance_sorted_idx][:num_features], height=0.7)
# ax1.set_yticklabels(feature_names[tree_importance_sorted_idx])
# ax1.set_yticks(tree_indices)
# ax1.set_ylim((0, len(model.feature_importances_)))
# ax2.boxplot(result.importances[perm_sorted_idx].T, vert=False,
#             labels=feature_names[perm_sorted_idx])
# fig.tight_layout()
# plt.show()

In [ ]:
# def score_model(model, X_train, y_train, X_test, y_test):
#     return {
#         'train_r2':model.score(X_train, y_train),
#         'train_rms3':np.sqrt(mean_squared_error(y_train, model.predict(X_train))),
#         'test_r2':model.score(X_test, y_test),
#         'test_rmse':np.sqrt(mean_squared_error(y_test, model.predict(X_test)))
#     }

In [ ]:
score_model(model, X_train, y_train, X_test, y_test)

# select K best features based on feature importance

In [ ]:
feature_selector = SelectFromModel(model, prefit=True)

In [ ]:
X_train_selected = feature_selector.transform(X_train)

In [ ]:
X_train_selected.shape

In [ ]:
len(feature_selector.get_support())

In [ ]:
selected_features = X_test.columns[feature_selector.get_support()]

In [ ]:
selected_features

In [ ]:
X_test_selected = X_test[selected_features]

In [ ]:
model.fit(X_train_selected, y_train)

In [ ]:
score_model(model, X_train_selected, y_train, X_test_selected, y_test)

this might be too many variable for a linear regression to handle. We might need to do some feature engineering to downsample these OR we can try using a model that will handle them better. but let's see how it does.

# plot residuals

In [ ]:
y_pred = model.predict(X_test_selected)

In [ ]:
plt.scatter(y_test, y_test-y_pred)

In [ ]:
y_test.min()

In [ ]:
y_test.max()

In [ ]:
plt.figure(figsize=(10,10))
plt.scatter(y_test, y_pred)
plt.title('Ammonia-N Prediction from Machine Learning Spectroscopy Inference Model')
plt.plot(np.linspace(0, 0.6, len(y_test)), np.linspace(0, 0.6, len(y_test)))
plt.xlabel('True Ammonia-N')
plt.ylabel('Predicted Ammonia-N')
plt.xlim(0,0.6)
plt.ylim(0,0.6)
# plt.savefig('ammonia_prediction_vs_truth.png')

In [ ]:
# regressor_OLS = OLS(endog = y_train, exog = X_train).fit()
# regressor_OLS.summary()

In [ ]:
# lets fill some nA

In [ ]:
# lr = LinearRegression()

In [ ]:
# lr.fit(X, y)